<p float="left">
  <img src="images/arrow_logo.jpg" alt="Arrow Logo" width="200" height="100" align="left"/>
  <img src="images/adi_logo.png" alt="ADI Logo" width="200" height="100" align="right"/>
</p>

<hr style="height:5px; text-align:bottom">

# Run The Cells Below
Run the cells below in sequence from top to bottom. Upon running the cell which initializes the required classes, a control widget and a pop-up containing data plots will appear.

To execute a cell:
1. Select the cell you wish to run by clicking the white space surrounding it. A blue border will appear around the cell.

    <img src="images/jupyter_cell_selected.PNG" alt="Jupyter Cell Selected"/>

2. Click the <strong>Run</strong> button at the top of the notebook to run the selected cell. Alternatively, you can use the CTRL+Enter keyboard combination to run the selected cell.
    - You can run the cells with markdown code in them, they have no effect on the program
    
3. After the cell has been run, a number indicating the current execution step will be inserted between the brackets on the left side of the cell.

    <img src="images/jupyter_cell_executed.PNG" alt="Jupyter Cell Executed"/>
    
    - Occasionally, after running the cell an asterisk may appear instead of a number, this indicates the cell is busy. Simply run the cell again to force/continue code execution.

    <img src="images/jupyter_cell_busy.PNG" alt="Jupyter Cell Busy"/>


If you make a mistake or run into problems, click <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page and start again. This clears the kernel of any variables or functions that may have been created in the previous program.

<hr style="height:1px">

## Running Test Mode

If you would like to run the test mode, which enables all of the channels, follow the steps below

1.Restart the kernel and clear the output by clicking <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page. This clears the kernel of any variables or functions that may have been created in the previous program.

2.First we need to import the Serial and Data modules required for the demo. We will also set the Matplotlib backend to provide a pop-up window for our data plots

In [ ]:
from safari_serial import Serial
from safari_data import Data
import time

import ipywidgets as widgets
from ipywidgets import Label, FloatText, Layout, Box, HTML
import threading

# Set the TkAgg backend for matplotlib
%matplotlib tk

3.We need to create a 'channels' data structure for enabling all of the AD7124 channels and storing all of the data.

In [ ]:
# Create a dictionary containing all of the enabled channels
# since we know all channels are enabled in test mode we can quickly create the dictionary
channels = {}
for i in range (16):
    channel = str(i)
    channels[channel] = {}
    # Number of elements to plot
    channels[channel]["max_elements"] = 300
    # Array for storing timestamps of when data was received
    channels[channel]["timestamps"]=[]
    # Array for storing channel voltages
    channels[channel]["voltages"]=[]
    # Array for storing the converted sensor values i.e. Temperature (deg. C) or Acceleration (G's)
    channels[channel]["values"]=[]
    # The axis object used for plotting. Each channel will get its own axis
    channels[channel]["axis"] = None
    # The annotation object used for displaying key values such as average, min, and max values for each channel
    channels[channel]["annotation"] = None
    # Boolean indicating if the annotation is visible. Default value is false since there are initially no values to annotate
    channels[channel]["annotation_visible"] = False
    # Text displayed by the annotation
    channels[channel]["annotation_text"] = ""
    # Line object used for the annotation
    channels[channel]["line"] = None
    # X-Y coordinates of the annotation box
    channels[channel]["annot_ax_xy"] = None
    # Color of the plot, default value is Black ('k')
    channels[channel]["axes_color"] = 'k'
    # Title of the plot
    channels[channel]["plot_title"] = f"Channel {channel}"
    # Y label of the plot when the voltages are displayed
    channels[channel]["y_label"] = ""
    # Y label of of the plot when values are displayed
    channels[channel]["y_label_converted"] = ""
    # Function for formatting voltage to plot, if None is specified, default operation is rounding voltage
    # to 4 decimal places: round(float(voltage), 4)
    channels[channel]["voltage_format_func"] = None
    # Function for converting voltage to a sensor value for plotting, if None is specified, default operation is 
    # rounding voltage to 4 decimal places: round(float(voltage), 4)
    channels[channel]["value_conversion_func"] = None

4.Now we will initialize the Serial class with the newly created data structure along with the desired configuration file so we can connect to the board and begin collecting data.

We will also create some simple text widgets to display the channel voltages. These widgets will be update using a thread so we don't interfere with the serial communication used to read the data.

After you run the cell you will be prompted to select the COM port that is connected to the Meerkat. Please refer to the [Identify COM Port](https://github.com/ArrowElectronics/arrow-safari/wiki/Running-Demo-in-Jupyter-Notebook#2-identify-com-port) section of the wiki for instructions.

<span style='color:red'><strong>Please note that the configuration file will be written to the board once the connect button is clicked.</strong></span>

In [ ]:
#Initialize the classes with the dictionary containing the channel information
serial = Serial(channels, 'ad7124_test_config.py')

for channel in channels:
    channels[channel]["label_widget"] = Label(value=f"Channel {channel} Voltage: ")
    channels[channel]["value_widget"] = HTML(value=f"<b><font color='red'>N/A</b>")
    display(widgets.HBox([channels[channel]["label_widget"], channels[channel]["value_widget"]]))

def update_test_values():
    while True:
        for channel in channels:
            if len(channels[channel]["voltages"]) > 0:
                channels[channel]["value_widget"].value = f"<b><font color='green'>{channels[channel]['voltages'][-1]}V"
        time.sleep(0.1)
        
# Setup threading to continually read data from the serial port
update_thread = threading.Thread(target=update_test_values)
update_thread.daemon = True
update_thread.start()

6.Once you are finished running the demo, shutdown the kernel by clicking <strong>Kernel -> Shutdown</strong> in the toolbar at the top of the page.